In [ ]:
import numpy as np
import pandas as pd 
import os
import gc
import sys
import time
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, StratifiedKFold
import xgboost as xgb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error

from transformers import AutoModel, AutoTokenizer
import json
from tensorflow.keras.models import load_model
import re
import pandas as pd
import string
import keras
from sklearn.svm import SVR



from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM,Dropout,concatenate
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Activation, Embedding, LSTM,Dropout,Bidirectional,GRU
from keras.utils import plot_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Flatten ,Embedding,Input,Conv1D,GlobalAveragePooling1D,GlobalMaxPooling1D,Dropout,MaxPooling1D,Bidirectional,GRU,Concatenate
from keras.models import Sequential,Model
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
import math

In [ ]:
data_dir = '../input/commonlitreadabilityprize/'
train = pd.read_csv(data_dir + 'train.csv')
test = pd.read_csv(data_dir + 'test.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

target = train['target'].to_numpy()

In [ ]:
def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

In [ ]:
train

In [ ]:
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['excerpt'] = train['excerpt'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['excerpt'] = test['excerpt'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
train['excerpt'] = train['excerpt'].str.lower()
test['excerpt'] = test['excerpt'].str.lower()

# remove numbers
train['excerpt'] = train['excerpt'].str.replace("[0-9]", " ")
test['excerpt'] = test['excerpt'].str.replace("[0-9]", " ")

In [ ]:
train['excerpt'] = train['excerpt'].apply(lambda x:' '.join(x.split()))
test['excerpt'] = test['excerpt'].apply(lambda x:' '.join(x.split()))

In [ ]:
train

In [ ]:
pip install "tensorflow>=1.7.0"

In [ ]:
!pip install tensorflow-hub

In [ ]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
tf.disable_eager_execution()

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
import pickle

In [ ]:
list_train = [train[i:i+10] for i in range(0,train.shape[0],10)]
list_test = [test[i:i+10] for i in range(0,test.shape[0],10)]
elmo_train = [elmo_vectors(x['excerpt']) for x in list_train]
elmo_test = [elmo_vectors(x['excerpt']) for x in list_test]
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)
pickle_out = open("elmo_train_v1","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

pickle_out = open("elmo_test_v1","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:
list_train = [train[i:i+20] for i in range(0,train.shape[0],20)]
list_test = [test[i:i+20] for i in range(0,test.shape[0],20)]
elmo_train = [elmo_vectors(x['excerpt']) for x in list_train]
elmo_test = [elmo_vectors(x['excerpt']) for x in list_test]
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)
pickle_out = open("elmo_train_v2","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

pickle_out = open("elmo_test_v2","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:
list_train = [train[i:i+30] for i in range(0,train.shape[0],30)]
list_test = [test[i:i+30] for i in range(0,test.shape[0],30)]
elmo_train = [elmo_vectors(x['excerpt']) for x in list_train]
elmo_test = [elmo_vectors(x['excerpt']) for x in list_test]
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)
pickle_out = open("elmo_train_v3","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

pickle_out = open("elmo_test_v3","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:
list_train = [train[i:i+40] for i in range(0,train.shape[0],40)]
list_test = [test[i:i+40] for i in range(0,test.shape[0],40)]
elmo_train = [elmo_vectors(x['excerpt']) for x in list_train]
elmo_test = [elmo_vectors(x['excerpt']) for x in list_test]
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)
pickle_out = open("elmo_train_v4","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

pickle_out = open("elmo_test_v4","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:
list_train = [train[i:i+50] for i in range(0,train.shape[0],50)]
list_test = [test[i:i+50] for i in range(0,test.shape[0],50)]
elmo_train = [elmo_vectors(x['excerpt']) for x in list_train]
elmo_test = [elmo_vectors(x['excerpt']) for x in list_test]
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)
pickle_out = open("elmo_train_v5","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

pickle_out = open("elmo_test_v5","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()